In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
!pip install pytorch-transformers
!pip install transformers
!pip install nltk


     |████████████████████████████████| 184kB 7.5MB/s 
     |████████████████████████████████| 890kB 11.7MB/s 
     |████████████████████████████████| 1.2MB 21.9MB/s 
     |████████████████████████████████| 133kB 51.4MB/s 
     |████████████████████████████████| 7.3MB 28.4MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=df640fde9ee93c8845f9a338d4a1964ecb57a86ff5d56182e312300f41897e0a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.27 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 1.9MB 8.1MB/s 
     |████████████████████████████████| 3.2MB 53.4MB/s 


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
!pip install tweet-preprocessor

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import DebertaTokenizer, DebertaModel, DebertaForSequenceClassification
from transformers import AdamW
import nltk
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# % matplotlib inline


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [7]:
MAX_LEN = 128

In [8]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/ISOT/dataset/Train_Test/TrainTest.csv')
train=train.iloc[0:16000]

In [9]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 9600
Size of validation set: 3200
Size of testing set: 3200


In [10]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()


In [12]:
p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess(row, lemmatizer, stemmer):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  tokenization = nltk.word_tokenize(text)     
  tokenization = [w for w in tokenization if not w in stop_words]
#   text = ' '.join([porter_stemmer.stem(w) for w in tokenization])
#   text = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [13]:
df['text'] = df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
val_df['text'] = val_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)
test_df['text'] = test_df.apply(lambda x: preprocess(x, wordnet_lemmatizer, porter_stemmer), 1)


In [14]:
def map_label(row):
  return 0 if row['label']==1 else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [15]:
train_sentences = df.text.values
val_sentences = val_df.text.values
test_sentences = test_df.text.values

train_labels = df.label_encoded.values
val_labels = val_df.label_encoded.values


In [ ]:
# tokenizer_sentences = np.array(list(df.tweet.values) + list(val_df.tweet.values))

In [16]:
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base', do_lower_case=True)


In [17]:
def Encode_TextWithAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=True):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids'],encoded_dict['attention_mask']

def Encode_TextWithoutAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=False):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids']

def get_TokenizedTextWithAttentionMask(sentenceList, tokenizer):
    token_ids_list,attention_mask_list = [],[]
    for sentence in sentenceList:
        token_ids,attention_mask = Encode_TextWithAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
        attention_mask_list.append(attention_mask)
    return token_ids_list,attention_mask_list

def get_TokenizedText(sentenceList, tokenizer):
    token_ids_list = []
    for sentence in sentenceList:
        token_ids = Encode_TextWithoutAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
    return token_ids_list

In [18]:
train_token_ids,train_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(train_sentences,tokenizer))
val_token_ids,val_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(val_sentences,tokenizer))
test_token_ids,test_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(test_sentences,tokenizer))

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [19]:
batch_size = 32

train_data = TensorDataset(train_token_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_token_ids, val_attention_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_token_ids, test_attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [20]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2).cuda()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base

In [21]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [22]:
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [23]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
train_loss_set = []
best_val_accuracy = 0.90
directory_path =  '/content/gdrive/MyDrive/4th_sem/dataset/ISOT/dataset/Train_Test'
epochs = 15

for _ in trange(epochs, desc="Epoch"):
  
  model.train()
  
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    loss.backward()
    optimizer.step()
        
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
  model.eval()

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  Validation_Accuracy = (eval_accuracy/nb_eval_steps)
  if(Validation_Accuracy >= best_val_accuracy):
    torch.save(model.state_dict(), directory_path+'/models/DeBERTa_base_best_model.ckpt')
    best_val_accuracy = Validation_Accuracy
    print('Model Saved')

    


Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Train loss: 0.04206706400747256
Validation Accuracy: 0.9965625


Epoch:   7%|▋         | 1/15 [04:53<1:08:26, 293.35s/it]

Model Saved
Train loss: 0.00419569427035943
Validation Accuracy: 0.9990625


Epoch:  13%|█▎        | 2/15 [09:49<1:03:42, 294.06s/it]

...

Model Saved
Train loss: 8.47045835598692e-06
Validation Accuracy: 0.9990625


Epoch:  93%|█████████▎| 14/15 [1:08:54<04:55, 295.53s/it]

Model Saved
Train loss: 7.611361686485907e-06
Validation Accuracy: 0.9990625


Epoch: 100%|██████████| 15/15 [1:13:49<00:00, 295.32s/it]

Model Saved


In [25]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLNetTokenizer, XLNetForSequenceClassification,BertTokenizer, BertForSequenceClassification, DebertaTokenizer, DebertaForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, ElectraTokenizer, ElectraForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [29]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/ISOT/dataset/Train_Test/TrainTest.csv')

In [30]:
train = train.rename(columns={ 'Unnamed: 0' : 'id'})
train=train.iloc[0:15000] 

In [31]:
train.head()

,id,title,text,subject,date,label
0,9831,Hero Church Member With CCL Stops Sudanese Sho...,The main stream media is totally ignoring the ...,politics,"Sep 24, 2017",0
1,14854,Gala glitz masks Asia's tensions as Trump wind...,MANILA (Reuters) - Leaders from across Asia jo...,worldnews,"November 12, 2017",1
2,4503,New York passes emergency spending plan to avo...,"ALBANY, N.Y. (Reuters) - New York lawmakers on...",politicsNews,"April 3, 2017",1
3,12050,"Honduran helicopter crash kills six, including...",TEGUCIGALPA (Reuters) - The sister of Honduran...,worldnews,"December 17, 2017",1
4,7623,Anti-Vaxxer Parents Let Toddler Die Of Mening...,A Canadian couple is facing charges after thei...,News,"March 8, 2016",0


In [32]:
df, val_df, test_df = np.split(train.sample(frac=1), [int(.6*len(train)), int(.8*len(train))])

df = df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
test_label_df = test_df

print("Size of training set: {}".format(len(df)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 9000
Size of validation set: 3000
Size of testing set: 3000


In [33]:
def preprocess(row):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [34]:
def map_label(row):
  return 0 if row['label']==1 else 1

df['label_encoded'] = df.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [35]:
train_sentences = df.text.values
train_token_ids = df.id.values
val_sentences = val_df.text.values
val_token_ids = val_df.id.values
test_sentences = test_df.text.values
test_token_ids = test_df.id.values


In [36]:
train_sentences = [sentence + " [SEP] [CLS]" for sentence in train_sentences]
train_labels = df.label_encoded.values
val_sentences = [sentence + " [SEP] [CLS]" for sentence in val_sentences]
val_labels = val_df.label_encoded.values
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]


In [37]:
MAX_LEN = 128
batch_size = 64


In [38]:
def get_dataloader(network, train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids):
  dic = {"XLNET": "xlnet-base-cased", "BERT": "bert-base-uncased", "ROBERT": "roberta-base", "XLM-ROBERT": "xlm-roberta-base", "DeBERTa": "microsoft/deberta-base", "Electra": "google/electra-base-discriminator"}
  if network=='XLNET':
    tokenizer = XLNetTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='DeBERTa':
    tokenizer = DebertaTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='BERT':
    tokenizer = BertTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='Electra':
    tokenizer = ElectraTokenizer.from_pretrained(dic[network], do_lower_case=True)
  else:
    tokenizer = RobertaTokenizerFast.from_pretrained(dic[network], do_lower_case=True)

  tokenized_train_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
  print ("Tokenize the first sentence:")
  print (tokenized_train_texts[0])

  tokenized_val_texts = [tokenizer.tokenize(sent) for sent in val_sentences]
  tokenized_test_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
  input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train_texts]
  input_val_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_val_texts]
  input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test_texts]

  input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_val_ids = pad_sequences(input_val_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Create attention masks
  train_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_train_ids:
    seq_mask = [float(i>0) for i in seq]
    train_attention_masks.append(seq_mask)
  train_masks = np.array(train_attention_masks)

  # Create attention masks
  val_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_val_ids:
    seq_mask = [float(i>0) for i in seq]
    val_attention_masks.append(seq_mask)
  validation_masks = np.array(val_attention_masks)

  # Create attention masks
  test_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_test_ids:
    seq_mask = [float(i>0) for i in seq]
    test_attention_masks.append(seq_mask)
  test_masks = np.array(test_attention_masks)

  train_inputs = torch.tensor(input_train_ids)
  validation_inputs = torch.tensor(input_val_ids)
  test_inputs = torch.tensor(input_test_ids)
  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(val_labels)
  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)
  test_masks = torch.tensor(test_masks)

  train_data = TensorDataset(torch.tensor(train_token_ids), train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(torch.tensor(val_token_ids), validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  test_data = TensorDataset(torch.tensor(test_token_ids), test_inputs, test_masks)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  return train_dataloader, validation_dataloader, test_dataloader


In [39]:
train_dataloader_DeBERTa, val_dataloader_DeBERTa, test_dataloader_DeBERTa = get_dataloader("DeBERTa", train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids)

Tokenize the first sentence:
['5661', '318', '331', '2217', '329', '19997', '290', '465', '5941', '287', '257', '1276', '12', '5404', '1181', '329', '597', '308', '404', '4540', '11', '428', '318', '262', '1266', '1705', '19997', '714', '3328', '3756', '656', '11752', '952', '264', '4165', '3071', '257', '1115', '12', '1014', '3234', '329', '262', '7930', '15265', '510', '329', '22378', '287', '262', '11752', '952', '41477', '4787', '4165', '318', '5922', '11', '290', '3308', '13', '1667', '1073', '6437', '952', '357', '81', '12', '2704', '8', '318', '319', '262', '2354', '286', '326', '3234', '2045', '287', '13', '272', '11752', '952', '4165', '3278', '3199', '256', '3322', '416', '627', '3732', '541', '9607', '2523', '836', '1940', '19997', '3756', '262', '835', '351', '3261', '1411', '1104', '11', '3940', '416', '1363', '1181', '467', '85', '13', '45610', '479', '292', '488', '11', '351', '2608', '1411', '13', '479', '292', '488', '318', '40060', '416', '3308', '13', '28501', '4630'

In [41]:
model4 = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
directory_path = '/content/gdrive/MyDrive/4th_sem/dataset/ISOT/dataset/Train_Test/models'
model4.load_state_dict(torch.load(directory_path+'/DeBERTa_base_best_model.ckpt'))
model4.eval()
model4.cuda()



Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'config', 'deberta.embeddings.position_embeddings.weight']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base

DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

In [42]:
def get_model_preds_labels(model, dataloader, mode='train'):
  data_vectors = []
  labels = []
  ids = []

  with torch.no_grad():
      correct = 0
      total = 0
      for i, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if mode=='test':
            token_ids, b_input_ids, b_input_mask = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            for point, tk_id in zip(F.softmax(outputs.logits).tolist(), token_ids.tolist()):
              data_vectors.append(point)       
              ids.append(tk_id)     
        else:
          # Unpack the inputs from our dataloader
          token_ids, b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          for point, lab, tk_id in zip(F.softmax(outputs.logits).tolist(), b_labels.tolist(), token_ids.tolist()):
            data_vectors.append(point)
            labels.append(lab)
            ids.append(tk_id)

  if mode=='test':
    return data_vectors, ids
  else:
    return data_vectors, labels, ids


#Train Data

In [43]:
m4, _, train_ids4 = get_model_preds_labels(model4, train_dataloader_DeBERTa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [44]:
pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in m4]
train_pred_df = pd.DataFrame({'id': train_ids4, 'predicted_label': pred_labels})
train_pred_df = df[["id", "label"]].merge(train_pred_df, on='id', how='left')
train_pred_df.head()

,id,label,predicted_label
0,14429,0,0
1,13653,1,1
2,12470,1,1
3,3056,0,0
4,16762,0,1


In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score


In [46]:
confusion_matrix(train_pred_df['label'].values, train_pred_df['predicted_label'].values)

array([[4658,  869],
       [ 821, 4300]])

In [48]:
print(accuracy_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values))
print(precision_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(recall_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(f1_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))


0.8412847483095417
0.8412847483095417
0.8412847483095417
0.8412847483095418


#Validation Data

In [50]:
v4, _, v_ids4 = get_model_preds_labels(model4, val_dataloader_DeBERTa)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [53]:
val_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in v4]
val_pred_df = pd.DataFrame({'id': v_ids4, 'predicted_label': val_pred_labels})
val_pred_df = val_df[["id", "label"]].merge(val_pred_df, on='id', how='left')
val_pred_df.head()



,id,label,predicted_label
0,10675,1,1
1,10876,0,0
2,11522,0,0
3,15654,1,1
4,20153,1,1


In [54]:
confusion_matrix(val_pred_df['label'].values, val_pred_df['predicted_label'].values)

array([[1568,  105],
       [  93, 1420]])

In [56]:
print(accuracy_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values))
print(precision_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(recall_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(f1_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))

0.9378531073446328
0.9378531073446328
0.9378531073446328
0.9378531073446328


#Test Data

In [58]:
t4, t_ids4 = get_model_preds_labels(model4, test_dataloader_DeBERTa, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [59]:
test_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in t4]
test_pred_df = pd.DataFrame({'id': t_ids4, 'predicted_label': test_pred_labels})
test_pred_df = test_label_df[["id", "label"]].merge(test_pred_df, on='id', how='left')
test_pred_df.head()

,id,label,predicted_label
0,981,0,0
1,12002,1,1
2,11563,1,1
3,14854,1,1
4,1197,0,0


In [60]:
confusion_matrix(test_pred_df['label'].values, test_pred_df['predicted_label'].values)

array([[1558,  125],
       [ 105, 1424]])

In [61]:
print(accuracy_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values))
print(precision_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))
print(recall_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))
print(f1_score(test_pred_df['label'].values, test_pred_df['predicted_label'].values, average='micro'))

0.9283935242839353
0.9283935242839353
0.9283935242839353
0.9283935242839353
